
# Project Level Solar Water Heating System Simulation Tool 

## Contents


* **Project input (minimum user requirment)**
* **Weather (CA climate zones available in the database)** 
* **Loads (example CA hourly loads available in the database)**
* **Performance paramters (defaults provided, user edits optional, such as providing insulation thickness for tanks)**
* **Component sizing (sizing rules provided, user edits optional, such as providing the actual length of the piping)**
* **System simulation examples with plots and summaries for all system configurations**

### System configurations
* Basecase gas tank water heater (WH)
* Active solar thermal system with a new tankless gas WH backup
* Active solar thermal system with a retrofit tankless gas WH backup


# Setup 
## Import modules 

In [1]:
import unittest
import os

os.chdir('../')

import numpy as np
import pandas as pd

from mswh.comm.label_map import SwhLabels
from mswh.system.components import Converter, Storage
from mswh.system.source_and_sink import SourceAndSink
from mswh.system.models import System
from mswh.system.source_and_sink import SourceAndSink

from mswh.tools.unit_converters import UnitConv, Utility
from mswh.comm.sql import Sql

from mswh.tools.plots import Plot
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import seaborn as sns
sns.set(rc={'figure.figsize':(16,12)})
sns.set_style("whitegrid")

import datetime

import logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)

# for repeatability
random_state = np.random.RandomState(123)

# label maps
c = SwhLabels().set_hous_labels()
s = SwhLabels().set_prod_labels()
r = SwhLabels().set_res_labels()

# pd.options.display.float_format = '{:,.3f}'.format

## Project inputs
* The user in the default case only needs to specify household occupancy and a climate in CA

In [2]:
# Add a value to the list for each additional household in a community scale project

# Community scale household occupancies for 4 households
occ_com = [2, 2, 2, 6]

# Individual scale household occupancy
occ_ind = [3]

# indicate whether any occupants are at home during the day
at_home_com = ['n', 'n', 'n', 'n']
at_home_ind = ['n']

# Example climate zones in CA
cold_cz = '01'
hot_cz = '15'
val_cz = '03'

## Load in input database (DB)

In [3]:
weather_db_path = os.path.join(os.getcwd(), 'mswh/comm/mswh_system_input.db')

db = Sql(weather_db_path)

try:
    # read table names for all tables in a
    # {table name : sheet name} form
    inputs = db.tables2dict(close = True)
except:
    msg = 'Failed to read input tables from {}.'
    log.error(msg.format(inpath))

## Generate weather data
* Includes calculation of the indcident irradiation with the default tilt and azimuth

In [4]:
source_and_sink = SourceAndSink(\
    input_dfs = inputs)

weather_cold = source_and_sink.irradiation_and_water_main( \
    cold_cz, method='isotropic diffuse')

weather_warm = source_and_sink.irradiation_and_water_main( \
    hot_cz, method='isotropic diffuse')

weather_val = source_and_sink.irradiation_and_water_main( \
    val_cz, method='isotropic diffuse')

INFO:mswh.system.source_and_sink:The user did not provide a numpy random state object. Initiating one with a provided or default seed value = 123.


## Select hourly hot water draw profiles from DB

In [5]:
# Returns hourly load profiles in m3/h and the peak load in gal/h for each household in the project
loads_com, peakload_com = \
    SourceAndSink._make_example_loading_inputs(\
    inputs, c, random_state, \
    occupancy = occ_com, at_home = at_home_com)

loads_indiv, peakload_indiv = \
    SourceAndSink._make_example_loading_inputs(\
    inputs, c, random_state, \
    occupancy = occ_ind, at_home = at_home_ind)

## Set performance parameters

* Active solar thermal system

In [6]:
# Community scale system (includes distribution pump parameters)

sol_the_sys_params_com = pd.DataFrame(data = \
            [[s['the_sto'], s['f_upper_vol'], .5], \
             [s['the_sto'], s['ins_thi'], .085], \
             [s['the_sto'], s['spec_hea_con'], .04], \
             [s['the_sto'], s['t_tap_set'], 322.04], \
             [s['the_sto'], s['h_vs_r'], 6.], \
             [s['the_sto'], s['dt_appr'], 2.], \
             [s['the_sto'], s['t_max_tank'], 344.15], \
             [s['the_sto'], s['eta_coil'], .84], \
             [s['the_sto'], s['circ'], 0.], \
             [s['sol_col'], s['interc_hwb'], .753], \
             [s['sol_col'], s['slope_hwb'], -4.025],\
             [s['sol_pump'], s['eta_sol_pump'], .85], \
             [s['piping'], s['pipe_spec_hea_con'], .0175], \
             [s['piping'], s['pipe_ins_thick'], .008], \
             [s['piping'], s['flow_factor'], .8], \
             [s['piping'], s['dia_len_exp'], 0.43082708345352605], \
             [s['piping'], s['dia_len_sca'], 0.007911283766743384],\
             [s['piping'], s['discr_diam_m'], \
               '[0.0127, 0.01905, 0.0254, '\
               '0.03175, 0.0381, 0.0508, 0.0635, 0.0762, 0.1016]'],\
             [s['piping'], s['circ'], False], \
             [s['piping'], s['long_br_len_fr'], 1.], \
             [s['dist_pump'], s['eta_dist_pump'], .85]], \
            columns = \
             [s['comp'], s['param'], s['param_value']])

sol_the_sys_params_com

,Component,Performance Parameter,Performance Parameter Value
0,thermal storage tank,upper volume fraction,0.5
1,thermal storage tank,insulation thickness,0.085
2,thermal storage tank,specific heat conductivity,0.04
3,thermal storage tank,tap temperature setpoint,322.04
4,thermal storage tank,height vs. radius,6
5,thermal storage tank,temperature difference (approach),2
6,thermal storage tank,maximum temperature,344.15
7,thermal storage tank,coil efficiency,0.84
8,thermal storage tank,circulation,0
9,solar collector,interc hwb,0.753


In [7]:
# Individual scale system, assuming a distribution pump is not needed similarly to a basecase gas tank WH

last_row_for_indiv = sol_the_sys_params_com.shape[0] - 1 - \
    sol_the_sys_params_com.loc[\
    sol_the_sys_params_com[s['comp']] == s['dist_pump'], :].\
    shape[0]

sol_the_sys_params_indiv = sol_the_sys_params_com.loc[:last_row_for_indiv,:]
sol_the_sys_params_indiv

sol_the_sys_params_indiv

,Component,Performance Parameter,Performance Parameter Value
0,thermal storage tank,upper volume fraction,0.5
1,thermal storage tank,insulation thickness,0.085
2,thermal storage tank,specific heat conductivity,0.04
3,thermal storage tank,tap temperature setpoint,322.04
4,thermal storage tank,height vs. radius,6
5,thermal storage tank,temperature difference (approach),2
6,thermal storage tank,maximum temperature,344.15
7,thermal storage tank,coil efficiency,0.84
8,thermal storage tank,circulation,0
9,solar collector,interc hwb,0.753


* Basecase and backup gas tank water heater (WH)

In [8]:
gas_tank_wh_params = pd.DataFrame(data = \
    [[s['gas_tank'], s['tank_re'], .78, '-'], \
     [s['gas_tank'], s['ins_thi'], .03, 'm'], \
     [s['gas_tank'], s['spec_hea_con'], .081, 'W/mK'], \
     [s['gas_tank'], s['t_tap_set'], 322.04, 'K']], \
    columns = \
     [s['comp'], s['param'], s['param_value'], s['param_unit']])

gas_tank_wh_params

,Component,Performance Parameter,Performance Parameter Value,Performance Parameter Unit
0,gas tank WH,tank recovery efficiency,0.780,-
1,gas tank WH,insulation thickness,0.030,m
2,gas tank WH,specific heat conductivity,0.081,W/mK
3,gas tank WH,tap temperature setpoint,322.040,K


* Gas tankless WH

In [9]:
gas_tankless_wh_params = pd.DataFrame(data = [[s['gas_burn'], s['comb_eff'], .85, '-']], \
    columns = [s['comp'], s['param'], s['param_value'], s['param_unit']])
gas_tankless_wh_params

,Component,Performance Parameter,Performance Parameter Value,Performance Parameter Unit
0,gas burner,combustion efficiency,0.85,-


* All performance parameters

In [10]:
# this input table provides all available component performance parameters
inputs['component_performance']

,Component ID,Performance Parameter,Performance Parameter Value,Performance Parameter Unit
0,1,interc hwb,0.753,-
1,1,slope hwb,-4.025,W/m2K
2,1,interc cd,0.75,-
3,1,a1 cd,-3.688,W/m2K
4,1,a2 cd,-0.0055,W/m2K2
5,4,PV efficiency,0.16,-
6,4,fraction of active PV area,1,-
7,4,reference irradiation,1000,W/m2
8,5,tank recovery efficiency,0.78,-
9,5,tap temperature setpoint,322.04,K


In [11]:
inputs['sys_3_components']

,Component ID,Component,Component Technology,Component Size Unit
0,1,solar collector,flat plate,m2
1,2,solar collector,tubular,m2
2,3,pv,monochrystalline,W
3,4,pv,polychrystalline,W
4,5,gas tank WH,conventional gas tank water heater,m3
5,6,electric resistance tank WH,conventional electric resistance tank water he...,m3
6,7,thermal storage tank,thermal storage tank with an in-tank coil,m3
7,8,gas burner,gas burner,W
8,9,electric resistance heater,in-tank or instantaneous,W
9,10,heat pump,in-tank,W


In [12]:
to_doc = inputs['component_performance'].merge(inputs['sys_3_components'], on = "Component ID", how = "left")

In [13]:
to_doc.to_clipboard()

## Component sizing

Note that backup is sized to full load.

* Gas tank WH: DOE sizing rule based on peak hourly demand +-2 gal

In [14]:
# Basecase gas tank WH

# in m3
gas_tank_size_indiv = UnitConv(peakload_indiv.loc[0, c['max_load']]).m3_gal(unit_in = 'gal')
gas_tank_size_indiv

# Individual household, backup
# same as the basecase sizing

0.158987304

In [15]:
# Community scale, backup

# get peak loads for each household
peakload_com['gas_tank_size_com'] = peakload_com[\
    c['max_load']].apply(lambda x: UnitConv(x).m3_gal(unit_in = 'gal'))
peakload_com.index = peakload_com[c['id']]

* Active solar thermal system: CSI Handbook sizing rules

In [16]:
# CSI demand estimate
def demand_estimate(occ):
    if occ == 1:
        return 20.
    if occ == 2:
        return 35.
    else:
        return 35.+10.*(occ-2.)
    
col_area_scaler = 1.2 #(CSI sizing rule: upper limit 1.25)
tank_vol_scaler = 1.3 #(CSI sizing rule: lower limit 1.25)

# Individual household (without a distribution pump)

# get demand estimate
demand_estimate_ind = demand_estimate(occ_ind[0])

# CSI sizing rules
col_area_ind_sqft = demand_estimate_ind * col_area_scaler
tank_vol_ind_gal = col_area_ind_sqft * tank_vol_scaler

col_area_ind = UnitConv(col_area_ind_sqft).sqft_m2()
tank_vol_ind = UnitConv(tank_vol_ind_gal).m3_gal(unit_in = 'gal')

# Community scale (with a distribution pump)

# get demand estimate
demand_estimate_com = 0.
for inx, row in loads_com.iterrows():
    demand_estimate_com += demand_estimate(row[c['occ']])

col_area_com_sqft = demand_estimate_com * col_area_scaler
tank_vol_com_gal = col_area_com_sqft * tank_vol_scaler

col_area_com = UnitConv(col_area_com_sqft).sqft_m2()
tank_vol_com = UnitConv(tank_vol_com_gal).m3_gal(unit_in = 'gal')

* Gas tankless WH: Based on the WH efficiency standards analysis and input powers collected online

In [17]:
def gas_tankles_size_W(occupancy):
    size = 24875.*occupancy**0.5175
    return size

* Solar pump

In [18]:
# solar pump
solar_pump_w_indiv = 7.5101 * sum(occ_ind) ** 0.5322
solar_pump_w_com = 7.5101 * sum(occ_com) ** 0.5322

### Distribution system

* Piping

In [19]:
# If the length of community piping installation is unknown, use this estimate

pipe_m_per_hhld = 3.048
# household type pipe length scalers for communities with attached and detached single family houses
multifamily = 1.
attached_k = 3.
detached_k = 6.

diameter_scaler=0.00791128
diameter_exponent=0.430827

# The average diameter of the distribution piping is a function of distribution pipe length
# d_m = diameter_scaler * (pipe_m_per_hhld * number_of_households)**diameter_exponent
# To change these parameters, edit values in sol_the_sys_params_{com/indiv} dataframes

* Distribution pump

In [20]:
dist_pump_w = 10.4376 * len(occ_com) ** 0.9277

## Compile component sizes for system simulation

* Basecase gas tank WH

In [21]:
# Individual household

gas_tank_wh_size = pd.DataFrame(
    data = [[s['gas_tank'], gas_tank_size_indiv, 'm3']], columns = [s['comp'], s['cap'], s['cap_unit']])

gas_tank_wh_size

,Component,Component Size,Component Size Unit
0,gas tank WH,0.158987,m3


* Solar thermal system, individual household

In [22]:
# Main system

sys_sizes_indiv = pd.DataFrame(data = \
    [[s['sol_col'], col_area_ind, 'm2'], \
     [s['the_sto'], tank_vol_ind, 'm3'], \
     [s['sol_pump'], solar_pump_w_indiv, 'W'], \
     [s['piping'], pipe_m_per_hhld, 'm']], \
            columns = \
             [s['comp'], s['cap'], s['cap_unit']])

sys_sizes_indiv

,Component,Component Size,Component Size Unit
0,solar collector,5.016764,m2
1,thermal storage tank,0.265736,m3
2,solar pump,13.476268,W
3,piping,3.048000,m


In [23]:
# Retrofit: Backup gas tank WH

retrofit_bckp_size_indiv = pd.DataFrame(data = \
            [[1, s['gas_tank'], gas_tank_size_indiv, 'm3']], columns = [c['id'], s['comp'], s['cap'], s['cap_unit']])
demand_estimate_com

180.0

In [24]:
# New: Backup gas tankless WH

new_bckp_size_ind = pd.DataFrame(data = [[1, s['gas_burn'], gas_tankles_size_W(occ_ind[0]), 'W']], \
            columns = [c['id'], s['comp'], s['cap'], s['cap_unit']])

new_bckp_size_ind

,Consumer ID,Component,Component Size,Component Size Unit
0,1,gas burner,43921.113177,W


* Solar thermal system, community

In [25]:
# Main system components

sys_sizes_com = pd.DataFrame(data = \
            [[s['sol_col'], col_area_com, 'm2'], \
             [s['the_sto'], tank_vol_com, 'm3'], \
             [s['sol_pump'], solar_pump_w_com, 'W'], \
             [s['dist_pump'], dist_pump_w, 'W'], \
             [s['piping'], multifamily * pipe_m_per_hhld * len(occ_com),'m']], \
            columns = \
             [s['comp'], s['cap'], s['cap_unit']])

sys_sizes_com

,Component,Component Size,Component Size Unit
0,solar collector,20.067057,m2
1,thermal storage tank,1.062944,m3
2,solar pump,28.182919,W
3,distribution pump,37.768671,W
4,piping,12.192000,m


In [26]:
# Retrofit: Backup gas tank WHs

retrofit_bckp_size_com = pd.DataFrame(columns = [c['id'], s['comp'], s['cap']],
                                      index = peakload_com.index)

for i in peakload_com.index:
    retrofit_bckp_size_com.loc[i, c['id']] = i
    retrofit_bckp_size_com.loc[i, s['comp']] = s['gas_tank']
    retrofit_bckp_size_com.loc[i, s['cap']] = peakload_com.loc[i, 'gas_tank_size_com'].round(3)

retrofit_bckp_size_com.index = range(retrofit_bckp_size_com.shape[0])
retrofit_bckp_size_com[s['cap_unit']] = 'm3'
retrofit_bckp_size_com

,Consumer ID,Component,Component Size,Component Size Unit
0,1,gas tank WH,0.139,m3
1,2,gas tank WH,0.179,m3
2,3,gas tank WH,0.138,m3
3,4,gas tank WH,0.139,m3


In [27]:
# New: Backup gas tankless WHs

new_bckp_size_com = pd.DataFrame(columns = [c['id'], s['comp'], s['cap']], index = peakload_com.index)

for i in peakload_com.index:
    new_bckp_size_com.loc[i, c['id']] = i
    new_bckp_size_com.loc[i, s['comp']] = s['gas_burn']
    new_bckp_size_com.loc[i, s['cap']] = gas_tankles_size_W(occ_com[i - 1])

new_bckp_size_com.index = range(new_bckp_size_com.shape[0])

new_bckp_size_com

,Consumer ID,Component,Component Size
0,1,gas burner,35607.9
1,2,gas burner,35607.9
2,3,gas burner,35607.9
3,4,gas burner,62871.9


* Validation sizing
Sizes based on several observed rated systems from OG-300 with an 80 gal tank and gas tankless backup
(https://secure.solar-rating.org/Certification/Ratings/RatingsReport.aspx?device=6926&units=METRICS)


In [28]:
# Solar thermal with tankless backup
sys_sizes_val = pd.DataFrame(data = \
            [[s['sol_col'], 3.9], \
             [s['the_sto'], UnitConv(80.).m3_gal(unit_in ='gal')], \
             [s['sol_pump'], solar_pump_w_indiv], \
             [s['piping'], 0.]], \
            columns = \
             [s['comp'], s['cap']])

sys_sizes_val

,Component,Component Size
0,solar collector,3.900000
1,thermal storage tank,0.302833
2,solar pump,13.476268
3,piping,0.000000


In [37]:
# visualisation setup
pd.options.display.float_format = '{:,.1f}'.format
w = 650
h = 450
f = 16
m_l = 50.
m_b = 50.

# hours of the year plotted time, a week in July starting on Monday
first_hour = 29*7*24 + 1
last_hour = first_hour + 7*24
first_hour

4873

# System Simulation
## Basecase gas tank WH

In [38]:
# cold climate

conv_gas_tank_wh_cold = System(sys_params = gas_tank_wh_params, \
            sys_sizes = gas_tank_wh_size, \
            weather = weather_cold, \
            loads = loads_indiv)

cons_total_cold, proj_total_cold, ts_proj_cold = conv_gas_tank_wh_cold.conventional_gas_tank()

# warm climate

conv_gas_tank_wh_warm = System(sys_params = gas_tank_wh_params, \
            sys_sizes = gas_tank_wh_size, \
            weather = weather_warm, \
            loads = loads_indiv)

cons_total_warm, proj_total_warm, ts_proj_warm = conv_gas_tank_wh_warm.conventional_gas_tank()

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.
INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Gas tank WH (WHAM) is set up.


In [39]:
data_bc = pd.DataFrame(data = \
        [ts_proj_cold[r['gas_use']], \
         ts_proj_warm[r['gas_use']]]).transpose()

fig = Plot(data_headers = [r['gas_use']+' CZ'+cold_cz,r['gas_use']+' CZ'+hot_cz], \
                 outpath = '', save_image = False, title = 'Basecase gas tank WH', label_h = 'Time [h]', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.01, legend_y = 1.).\
            series(data_bc[first_hour: last_hour], outfile = '', index_in_a_column = None, modes = 'lines')

## Individual new solar thermal system

In [40]:
# in a cold climate
sol_wh_indiv_new_cold = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_indiv,
                          backup_sizes = new_bckp_size_ind,
                          weather = weather_cold,
                          loads = loads_indiv)

cons_total_indiv_new_cold, proj_total_indiv_new_cold,\
[proj_total_dict_indiv_new_cold, sol_fra_indiv_new_cold, pump_el_use_indiv_new_cold,\
 pump_op_hour_indiv_new_cold, ts_res_indiv_new_cold, backup_ts_cons_indiv_new_cold, rel_err_indiv_new_cold] = \
    sol_wh_indiv_new_cold.solar_thermal(backup = 'gas')

INFO:mswh.system.models:Assigned weather data timeseries.
INFO:mswh.system.components:Assigned weather data timeseries.
INFO:mswh.system.components:Solar storage tank is set.
INFO:mswh.system.components:Assigned weather data timeseries.


In [41]:
print('month, day, hour:\nData starts at', ts_res_indiv_new_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_new_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'CZ' + cold_cz, label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.05, legend_y = 1.2).\
            series(ts_res_indiv_new_cold.loc[first_hour : last_hour,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

month, day, hour:
Data starts at [7 23 0] and ends at [7 30 0]


In [ ]:
fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = 'Solar tank, tankless backup, individual household, CZ' + cold_cz, label_v = 'Temperature [K]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_new_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

In [ ]:
# in a warm climate
sol_wh_indiv_new_warm = System(sys_params = sol_the_sys_params_indiv, \
                          backup_params = gas_tankless_wh_params, \
                          sys_sizes = sys_sizes_indiv, \
                          backup_sizes = new_bckp_size_ind, \
                          weather = weather_warm, \
                          loads = loads_indiv)

cons_total_indiv_new_warm, proj_total_indiv_new_warm,\
[proj_total_dict_indiv_new_warm, sol_fra_indiv_new_warm, pump_el_use_indiv_new_warm,\
 pump_op_hour_indiv_new_warm, ts_res_indiv_new_warm, backup_ts_cons_indiv_new_warm, rel_err_indiv_new_warm] = \
    sol_wh_indiv_new_warm.solar_thermal(backup = 'gas')

In [ ]:
# first_hour = 4897
# last_hour = 4897 + 5*24

print('month, day, hour:\nData starts at', ts_res_indiv_new_warm.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_new_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'CZ' + hot_cz, label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.05, legend_y = 1.2).\
            series(ts_res_indiv_new_warm.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

In [ ]:
fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = 'Solar tank, tankless backup, individual household, CZ' + hot_cz, label_v = 'Temperature [K]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_new_warm.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

### Compare individual new system annual totals for two climates

In [ ]:
# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(proj_total_indiv_new_cold)
df = df.transpose()
df.index = ['indiv, new, cold CZ' + cold_cz]
df1 = pd.DataFrame(proj_total_indiv_new_warm)
df1 = df1.transpose()
df1.index = ['indiv, new, warm CZ' + hot_cz]
compare_indiv_new = df.append(df1).transpose()
compare_indiv_new

## Community new solar thermal system

In [ ]:
# in a cold climate
sol_wh_com_new_cold = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = new_bckp_size_com,
                          weather = weather_cold,
                          loads = loads_com)

cons_total_com_new_cold, proj_total_com_new_cold,\
[proj_total_dict_com_new_cold, sol_fra_com_new_cold, pump_el_use_com_new_cold,\
 pump_op_hour_com_new_cold, ts_res_com_new_cold, backup_ts_cons_com_new_cold, rel_err_com_new_cold] = \
    sol_wh_com_new_cold.solar_thermal(backup = 'gas')

In [ ]:
# first_hour = 4897
# last_hour = 4897 + 5*24

print('month, day, hour:\nData starts at', ts_res_com_new_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_new_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'CZ' + cold_cz, label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.).\
            series(ts_res_com_new_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

In [ ]:
# in a warm climate
sol_wh_com_new_warm = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = new_bckp_size_com,
                          weather = weather_warm,
                          loads = loads_com)

cons_total_com_new_warm, proj_total_com_new_warm,\
[proj_total_dict_com_new_warm, sol_fra_com_new_warm, pump_el_use_com_new_warm,\
 pump_op_hour_com_new_warm, ts_res_com_new_warm, backup_ts_cons_com_new_warm, rel_err_com_new_warm] = \
    sol_wh_com_new_warm.solar_thermal(backup = 'gas')

In [ ]:
print('month, day, hour:\nData starts at', ts_res_com_new_warm.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_new_warm.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'CZ' + hot_cz, label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False,margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.3).\
            series(ts_res_com_new_warm.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

### Compare community new system annual totals for two climates

In [ ]:
# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(proj_total_com_new_cold)
df = df.transpose()
df.index = ['com, new, cold CZ' + cold_cz]
df1 = pd.DataFrame(proj_total_com_new_warm)
df1 = df1.transpose()
df1.index = ['com, new, warm CZ' + hot_cz]
compare_indiv_new = df.append(df1).transpose()
compare_indiv_new

In [ ]:
# Project level total vs. individual household total
print(abs(cons_total_com_new_warm["Energy Use - Gas"].sum() -
                    proj_total_com_new_warm["Energy Use - Gas"]).round(2))

print(abs(cons_total_com_new_warm["Tank Heat Delivered"].sum() -
                    proj_total_com_new_warm["Tank Heat Delivered"]).round(2))

### Compare new installation systems across climates and scales

In [ ]:
# for cold climate

# Energy use per household
com = cons_total_com_new_cold.sum()/4
ind = cons_total_indiv_new_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['per household, com, new, cold CZ' + cold_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, new, cold CZ' + cold_cz]
compare_cold = df.append(df1).transpose()

# for warm climate

# Energy use per household
com = cons_total_com_new_warm.sum()/4
ind = cons_total_indiv_new_warm.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['per household, com, new, warm CZ' + hot_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, new, warm CZ' + hot_cz]
compare_warm = df.append(df1).transpose()
compare = compare_cold.copy()
compare[compare_warm.columns] = compare_warm

compare

## Individual retrofit solar thermal system

In [ ]:
# in a cold climate
sol_wh_indiv_retrofit_cold = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_indiv,
                          backup_sizes = retrofit_bckp_size_indiv,
                          weather = weather_cold,
                          loads = loads_indiv)

cons_total_indiv_retrofit_cold, proj_total_indiv_retrofit_cold,\
[proj_total_dict_indiv_retrofit_cold, sol_fra_indiv_retrofit_cold, pump_el_use_indiv_retrofit_cold,\
 pump_op_hour_indiv_retrofit_cold, ts_res_indiv_retrofit_cold, backup_ts_cons_indiv_retrofit_cold, rel_err_indiv_retrofit_cold] = \
    sol_wh_indiv_retrofit_cold.solar_thermal(backup = 'retrofit')

In [ ]:
print('month, day, hour:\nData starts at', ts_res_indiv_retrofit_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_retrofit_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Solar tank, retrofit backup, individual household, CZ' + cold_cz, label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

In [ ]:
fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = 'Solar tank, retrofit backup, individual household, CZ' + cold_cz, label_v = 'Temperature [K]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

In [ ]:
# in a warm climate
sol_wh_indiv_retrofit_warm = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_indiv,
                          backup_sizes = retrofit_bckp_size_indiv,
                          weather = weather_warm,
                          loads = loads_indiv)

cons_total_indiv_retrofit_warm, proj_total_indiv_retrofit_warm,\
[proj_total_dict_indiv_retrofit_warm, sol_fra_indiv_retrofit_warm, pump_el_use_indiv_retrofit_warm,\
 pump_op_hour_indiv_retrofit_warm, ts_res_indiv_retrofit_warm, backup_ts_cons_indiv_retrofit_warm, rel_err_indiv_retrofit_warm] = \
    sol_wh_indiv_retrofit_warm.solar_thermal(backup = 'retrofit')

In [ ]:
print('month, day, hour:\nData starts at', ts_res_indiv_retrofit_warm.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_retrofit_warm.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Solar tank, retrofit backup, individual household, CZ' + hot_cz, label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.05, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_warm.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

In [ ]:
fig = Plot(data_headers = [r['t_tank_up'], r['t_tank_low']], \
                 outpath = '', save_image = False, title = 'Solar tank, retrofit backup, individual household, CZ' + hot_cz, label_v = 'Temperature [K]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_indiv_retrofit_warm.loc[first_hour + 1 : last_hour + 1,\
                [r['t_tank_up'], r['t_tank_low']]], \
                 outfile = '', modes = 'lines')

### Compare individual retrofit system annual totals for two climates

In [ ]:
# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(proj_total_indiv_retrofit_cold)
df = df.transpose()
df.index = ['indiv, retrofit, cold CZ' + cold_cz]
df1 = pd.DataFrame(proj_total_indiv_retrofit_warm)
df1 = df1.transpose()
df1.index = ['indiv, retrofit, warm CZ' + hot_cz]
compare_indiv_retrofit = df.append(df1).transpose()
compare_indiv_retrofit

## Community retrofit solar thermal system

In [ ]:
# in a cold climate
sol_wh_com_retrofit_cold = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = retrofit_bckp_size_com,
                          weather = weather_cold,
                          loads = loads_com)

cons_total_com_retrofit_cold, proj_total_com_retrofit_cold,\
[proj_total_dict_com_retrofit_cold, sol_fra_com_retrofit_cold, pump_el_use_com_retrofit_cold,\
 pump_op_hour_com_retrofit_cold, ts_res_com_retrofit_cold, backup_ts_cons_com_retrofit_cold, rel_err_com_retrofit_cold] = \
    sol_wh_com_retrofit_cold.solar_thermal(backup = 'retrofit')

In [ ]:
print('month, day, hour:\nData starts at', ts_res_com_retrofit_cold.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_retrofit_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Solar tank, individual household, CZ' + hot_cz, label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.).\
            series(ts_res_com_retrofit_cold.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

In [ ]:
# in a warm climate
sol_wh_com_retrofit_warm = System(sys_params = sol_the_sys_params_com,
                          backup_params = gas_tank_wh_params,
                          sys_sizes = sys_sizes_com,
                          backup_sizes = retrofit_bckp_size_com,
                          weather = weather_warm,
                          loads = loads_com)

cons_total_com_retrofit_warm, proj_total_com_retrofit_warm,\
[proj_total_dict_com_retrofit_warm, sol_fra_com_retrofit_warm, pump_el_use_com_retrofit_warm,\
 pump_op_hour_com_retrofit_warm, ts_res_com_retrofit_warm, backup_ts_cons_com_retrofit_warm, rel_err_com_retrofit_warm] = \
    sol_wh_com_retrofit_warm.solar_thermal(backup = 'retrofit')

In [ ]:
print('month, day, hour:\nData starts at', ts_res_com_retrofit_warm.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_retrofit_warm.loc[last_hour, ['month','day', 'hour']].transpose().values)

fig = Plot(data_headers = [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']], \
                 outpath = '', save_image = False, title = 'Solar tank, individual household, CZ' + hot_cz, label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = False, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = 0.75, legend_y = 1.15).\
            series(ts_res_com_retrofit_warm.loc[first_hour + 1 : last_hour + 1,\
                [r['q_dem'], r['q_del_tank'],r['q_unmet_tank'],r['q_del_sol'],r['q_dist_loss']]], \
                 outfile = '', modes = 'lines')

### Compare community retrofit system annual totals for two climates

In [ ]:
# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(proj_total_com_retrofit_cold)
df = df.transpose()
df.index = ['indiv, retrofit, cold CZ' + cold_cz]
df1 = pd.DataFrame(proj_total_com_retrofit_warm)
df1 = df1.transpose()
df1.index = ['indiv, retrofit, warm CZ' + hot_cz]
compare_indiv_retrofit = df.append(df1).transpose()
compare_indiv_retrofit

In [ ]:
# Project level total vs. individual household total
print(abs(cons_total_com_retrofit_warm["Energy Use - Gas"].sum() -
                    proj_total_com_retrofit_warm["Energy Use - Gas"]).round(2))

print(abs(cons_total_com_retrofit_warm["Tank Heat Delivered"].sum() -
                    proj_total_com_retrofit_warm["Tank Heat Delivered"]).round(2))

## Compare systems across climates, configurations and scales

In [ ]:
# for cold climate

# Energy use per household
com = cons_total_com_retrofit_cold.sum()/4
ind = cons_total_indiv_retrofit_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['per household, com, retrofit, cold CZ' + cold_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, retrofit, cold CZ' + cold_cz]
compare_cold = df.append(df1).transpose()

# for warm climate

# Energy use per household
com = cons_total_com_retrofit_warm.sum()/4
ind = cons_total_indiv_retrofit_warm.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['per household, com, retrofit, warm CZ' + hot_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, retrofit, warm CZ' + cold_cz]
compare_warm = df.append(df1).transpose()
compare = compare_cold.copy()
compare[compare_warm.columns] = compare_warm

compare.drop(index='Consumer ID', axis = 1, inplace = True)
compare

In [ ]:
# New installations

# for cold climate

# Energy use per household
com = cons_total_com_new_cold.sum()/4
ind = cons_total_indiv_new_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['per household, com, new, cold CZ' + cold_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, new, cold CZ' + cold_cz]
compare_cold = df.append(df1).transpose()
compare = compare_cold.copy()

# for warm climate

# Energy use per household
com = cons_total_com_new_warm.sum()/4
ind = cons_total_indiv_new_warm.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['per household, com, new, warm CZ' + hot_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, new, warm CZ' + hot_cz]
compare_warm = df.append(df1).transpose()
compare[compare_warm.columns] = compare_warm


# Retrofits

# for cold climate

# Energy use per household
com = cons_total_com_retrofit_cold.sum()/4
ind = cons_total_indiv_retrofit_cold.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['per household, com, retrofit, cold CZ' + cold_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, retrofit, cold CZ' + cold_cz]
compare_cold_retr = df.append(df1).transpose()
compare[compare_cold_retr.columns] = compare_cold_retr

# for warm climate

# Energy use per household
com = cons_total_com_retrofit_warm.sum()/4
ind = cons_total_indiv_retrofit_warm.sum()

# temperatures are averaged, heat contents summed over a year period
df = pd.DataFrame(com)
df = df.transpose()
df.index = ['per household, com, retrofit, warm CZ' + hot_cz]
df1 = pd.DataFrame(ind)
df1 = df1.transpose()
df1.index = ['indiv, retrofit, warm CZ' + hot_cz]
compare_warm_retr = df.append(df1).transpose()
compare[compare_warm_retr.columns] = compare_warm_retr

subset = compare.drop(['Consumer ID'], axis = 0).loc[["Net Heat Demand", "Total Heat Delivered",
        "Energy Use - Gas","Energy Use - Gas, w/o Dist Loss",
        "Energy Use - Electricity", "Solar Fraction", "Backup Heat Delivered",
        "Unmet Heat", "Distribution Heat Loss"],:]
compare

In [ ]:
subset

### Duration curves for individual households gas use

In [ ]:
# climates: warm and cold
# configrations: retrofit and new

# single community system household

print('month, day, hour:\nData starts at', ts_res_com_retrofit_warm.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_com_retrofit_warm.loc[last_hour, ['month','day', 'hour']].transpose().values)

df_gas_use = pd.DataFrame()

df_gas_use["New installation, cold (CZ01), household 1"] = np.array(backup_ts_cons_com_new_cold[1][r['gas_use']])
df_gas_use["New installation, warm (CZ15), household 1"] = np.array(backup_ts_cons_com_new_warm[1][r['gas_use']])
df_gas_use["Retrofit, cold (CZ01), household 1"] = np.array(backup_ts_cons_com_retrofit_cold[1][r['gas_use']])
df_gas_use["Retrofit, warm (CZ15), household 1"] = np.array(backup_ts_cons_com_retrofit_warm[1][r['gas_use']])

fig = Plot(data_headers = None, \
                 outpath = '', save_image = False, title = '', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = True, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = .1, legend_y = 1.15).\
            series(df_gas_use, outfile = '', modes = 'lines')

In [ ]:
# climates: warm and cold
# configrations: retrofit and new

# individual system

print('month, day, hour:\nData starts at', ts_res_indiv_retrofit_warm.loc[first_hour, ['month','day', 'hour']].transpose().values, 'and ends at',
ts_res_indiv_retrofit_cold.loc[last_hour, ['month','day', 'hour']].transpose().values)

df_gas_use = pd.DataFrame()

df_gas_use["New installation, cold (CZ01), individual"] = np.array(backup_ts_cons_indiv_new_cold[1][r['gas_use']])
df_gas_use["New installation, warm (CZ15), individual"] = np.array(backup_ts_cons_indiv_new_warm[1][r['gas_use']])
df_gas_use["Retrofit, cold (CZ01), individual"] = np.array(backup_ts_cons_indiv_retrofit_cold[1][r['gas_use']])
df_gas_use["Retrofit, warm (CZ15), individual"] = np.array(backup_ts_cons_indiv_retrofit_warm[1][r['gas_use']])

fig = Plot(data_headers = None, \
                 outpath = '', save_image = False, title = '', label_v = 'Heat rate [W]',\
                 legend = True, duration_curve = True, margin_l = m_l, margin_b = m_b,\
                 notebook_mode = True, width = w, height = h, fontsize = f, legend_x = .1, legend_y = 1.15).\
            series(df_gas_use, outfile = '', modes = 'lines')

## Solar thermal system validation

We performed validation with average results for solar thermal systems with an instantaneous backup provided in OG-300 rating data. Using average savings and climate zone 3 solar fractions for all systems from above referenced file.

We also performed the component models validation. Those validation tests can be found in `swh.system.tests.test_components`.

In [ ]:
# scaled loads to match 300L/day, as used to obtain the rating data
load_array_val = loads_indiv['End-Use Load'][0] * \
    (300*365*.001)/loads_indiv['End-Use Load'][0].sum()

loads_indiv_val = pd.DataFrame(data = \
     [[1, occ_ind[0], load_array_val]],\
    columns = \
     [c['id'], c['occ'], c['load_m3']])

In [ ]:
# in a cold climate
sol_wh_indiv_new_val = System(sys_params = sol_the_sys_params_indiv,
                          backup_params = gas_tankless_wh_params,
                          sys_sizes = sys_sizes_val,
                          backup_sizes = new_bckp_size_ind,
                          weather = weather_val,
                          loads = loads_indiv_val)

cons_total_indiv_new_val, proj_total_indiv_new_val,\
[proj_total_dict_indiv_new_val, sol_fra_indiv_new_val, pump_el_use_indiv_new_val,\
 pump_op_hour_indiv_new_val, ts_res_indiv_new_val, backup_ts_cons_indiv_new_val, rel_err_indiv_new_val] = \
    sol_wh_indiv_new_val.solar_thermal(backup = 'gas')

In [ ]:
# compare solar fraction for CZ03
rating_sheet_03 = 0.69
average_from_rating_sheet_for_similar_installations_03 = 0.72
proj_total_indiv_new_val['Solar Fraction']

In [ ]:
# compare savings to basecase with the parameters corresponding the DOE appliance efficiency level 1
basecase_wh_val = System(sys_params = gas_tank_wh_params, \
            sys_sizes = gas_tank_wh_size, \
            weather = weather_val, \
            loads = loads_indiv_val)
bc_cons_total_el, bc_proj_total_el, bc_ts_proj_el = \
            basecase_wh_val.conventional_gas_tank()

In [ ]:
# compare savings (note that there is no information in the rating sheet on the basecase tank WH)

rating_sheet_03 = 4490
average_from_rating_sheet_for_similar_installations_03 = 4707.18

abs((average_from_rating_sheet_for_similar_installations_03 - ((bc_proj_total_el['Energy Use - Gas']
    - proj_total_indiv_new_val['Energy Use - Gas'])/1000))/average_from_rating_sheet_for_similar_installations_03).round(2)

In [ ]:
# With baseline tank with a slightly higher RE and insulation set to R12, as used for the solar tank
gas_tank_wh_params_val = pd.DataFrame(data = \
    [[s['gas_tank'], s['tank_re'], .82, '-'], \
     [s['gas_tank'], s['ins_thi'], .04, 'm'], \
     [s['gas_tank'], s['spec_hea_con'], .085, 'W/mK'], \
     [s['gas_tank'], s['t_tap_set'], 322.04, 'K']], \
    columns = \
     [s['comp'], s['param'], s['param_value'], s['param_unit']])
basecase_wh_val = System(sys_params = gas_tank_wh_params_val, \
            sys_sizes = gas_tank_wh_size, \
            weather = weather_val, \
            loads = loads_indiv_val)
bc_cons_total, bc_proj_total, bc_ts_proj = \
            basecase_wh_val.conventional_gas_tank()

# gas use savings compared to baseline
(bc_proj_total['Energy Use - Gas'] - proj_total_indiv_new_val['Energy Use - Gas'])/1000

In [ ]:
# Error
abs((average_from_rating_sheet_for_similar_installations_03 - ((bc_proj_total['Energy Use - Gas']
    - proj_total_indiv_new_val['Energy Use - Gas'])/1000))/average_from_rating_sheet_for_similar_installations_03)